# School Timetable Optimization Notebook

**Author:** Blessing Mvana Nhlozi  
**Objective:** Optimize a school timetable using OR-Tools CP-SAT solver for efficient scheduling of classes, teachers, and students.

## Methodology


**Variable and Model Definition**
   - Define decision variables `x[(class, day, period)]` to indicate whether a class is scheduled at a particular time.
   - Build CP-SAT model using constraints

**Constraint Programming**
   - **OR-Tools CP-SAT** solver was used to handle complex constraints.
   - Soft constraints were added to minimize gaps or distribute lessons evenly.

**Model Solving**
   - Solve the model using `CpSolver()`.
   - Check solution feasibility (`OPTIMAL` or `FEASIBLE`).

**Output Formatting**
   - Convert solution variables to a readable timetable (`DataFrame`).
   - Export timetable as `timetable.csv` with two columns: timeslot and class.

### Imports & Logging setup

In [1]:
from __future__ import annotations
import math, logging, time, tracemalloc
from typing import Dict, List, Tuple

import pandas as pd
from ortools.sat.python import cp_model

In [2]:
# Logging setup
logging.basicConfig(
    format="%(asctime)s | %(levelname)s | %(message)s",
    level=logging.INFO
)

### Load Functions


```python 
%run 000_Functions.ipynb
```
will load functions stored in  `000_Functions.ipynb` notebook located in the same folder. \
The functions for data loading, model building, solving, and output formatting.

In [3]:
%run 000_Functions.ipynb

### Main Execution Function

- Load all necessary data.
- Build the constraint model.
- Solve the timetable problem.
- Format and export the results.

In [4]:
def main():
    with Profiler("Data Loading"):
        class_list, timeslots_days, lessons_required, weekday, teacher_classes, student_classes, teacher_list = load_data()

    classes = class_list["class"].tolist()
    days = weekday["weekday"].tolist()
    periods = range(9)

    num_lessons = dict(zip(lessons_required["class"], lessons_required["num_lessons"]))
    class_teacher = dict(zip(teacher_classes["class"], teacher_classes["teacher"]))
    class_students = student_classes.groupby("class")["student"].apply(list).to_dict()

    with Profiler("Model Building"):
        model, x = build_model(classes, days, periods, num_lessons, class_teacher, class_students)

    with Profiler("Solving Model"):
        solution = solve_model(model, x)

    with Profiler("Output Formatting"):
        final = format_output(solution, timeslots_days)
        final.to_csv("timetable.csv", index=False)

    logging.info("Timetable successfully written to timetable.csv")

### Execute the Main Function

- Run the main function when this script is executed.
- This ensures that the notebook works both interactively and as a standalone script.

In [5]:
if __name__ == "__main__":
    main()

2025-10-06 16:00:27,196 | INFO | Starting: Data Loading
2025-10-06 16:00:27,240 | INFO | Finished: Data Loading | Time: 0.04s | Memory: 143.43 KB | Peak: 406.66 KB
2025-10-06 16:00:27,245 | INFO | Starting: Model Building
2025-10-06 16:00:34,270 | INFO | Finished: Model Building | Time: 7.02s | Memory: 11574.91 KB | Peak: 11944.07 KB
2025-10-06 16:00:34,272 | INFO | Starting: Solving Model
2025-10-06 16:00:45,389 | INFO | Finished: Solving Model | Time: 11.12s | Memory: 176.50 KB | Peak: 2151.84 KB
2025-10-06 16:00:45,391 | INFO | Starting: Output Formatting
2025-10-06 16:00:45,445 | INFO | Finished: Output Formatting | Time: 0.05s | Memory: 234.13 KB | Peak: 420.99 KB
2025-10-06 16:00:45,447 | INFO | Timetable successfully written to timetable.csv


## Notes

- The notebook is modular, separating functions from execution for clarity.
- CP-SAT solver strictly enforces constraints.
- Logging provides progress updates.